# Missing Data Imputation
Course: HUDK 4051

Author: Chendanni LIU

Assignment: ICE5

oBJ:
simulate MCAR data or MAR data
implement common missing data imputation methods
(optional) compare the performance of missing data imputation methods on an analysis you have done before.

# Missing data mechanism

MCAR: Missing complete at random - Whether or not  
Y
i
  is missing is independent of both  
Y
i
  and any observed covariate  
X
i
 .
MAR: Missing at Random - Whether or not  
Y
i
  is missing is independent of  
Y
i
  given observed covariates  
X
i
 .
NMAR (or sometimes MNAR): Not missing at random/Missing not at random - This is the most generic mechanism, where the cause of the missingness cannot be controlled. Whether or not  
Y
i
  is missing is not independent even after controlling the observed covariates  
X
i
 .

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
irisRaw = datasets.load_iris()

# Convert it to a pandas dataframe for better visualization
iris = pd.DataFrame(data=irisRaw.data, columns=irisRaw.feature_names)
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


# MCAR

In [3]:
iris_MCAR = iris.copy()

In [4]:
# Set the missingness rate at 30%
missing = 0.3

# Only introduce missing rate to the `sepal length (cm)`.
iris_MCAR.loc[iris_MCAR.sample(frac=missing).index, 'sepal length (cm)'] = np.nan

iris_MCAR

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,NaN,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
# Alternatively, iterate through all columns and introduce 20% missing to all variables.
for col in iris_MCAR.columns:
    iris_MCAR.loc[iris_MCAR.sample(frac=missing).index, col] = np.nan

# MAR

One easy way to generate an MAR data is to use the weights parameter in pd.sample(). The default ‘None’ results in equal probability weighting. If passed a Series, sample() will align with target object on index. Using a DataFrame column as weights, rows with larger value in the the specified column are more likely to be sampled. Read here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html

In [6]:
iris_MAR = iris.copy()

In [7]:
# Set the missingness rate at 30%
missing = 0.3

# Only introduce missing rate to the `sepal length (cm)`.
iris_MAR.loc[iris_MAR.sample(frac=missing, weights = 'sepal width (cm)').index, 'sepal length (cm)'] = np.nan

iris_MAR

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


# Deletion Methods

Listwise deletion (or sometimes known as casewise deletion)
Pairwise deletion

In [8]:
iris_MCAR_listwise = iris_MCAR.copy()
iris_MCAR_listwise.dropna()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1
26,5.0,3.4,1.6,0.4
31,5.4,3.4,1.5,0.4
32,5.2,4.1,1.5,0.1
36,5.5,3.5,1.3,0.2
41,4.5,2.3,1.3,0.3
52,6.9,3.1,4.9,1.5
55,5.7,2.8,4.5,1.3
57,4.9,2.4,3.3,1.0


# Pairwise deletion

Alternatively, we can do pairwise deletion. It only skips the null observation when the it is needed. Therefore, it avoid the data loss issue in listwise deletion to a certain degree. All methods in pandas such as mean, median, sum, etc. uses pairwise deletion by default. You can see that pairwise deletion performs a bit better on MCAR than MAR data (closer to the true estimation).

In [9]:
iris_MCAR_pairwise = iris_MCAR.copy()
iris_MAR_pairwise = iris_MAR.copy()

print("Mean estimation in MCAR data with pairwise deletion",
      iris_MCAR_pairwise['sepal length (cm)'].mean(), 
     "\nSD estimation in MCAR data with pairwise deletion",
      iris_MCAR_pairwise['sepal length (cm)'].std()
     )
print("Mean estimation in MAR data with pairwise deletion",
      iris_MAR_pairwise['sepal length (cm)'].mean(), 
     "\nSD estimation in MAR data with pairwise deletion",
      iris_MAR_pairwise['sepal length (cm)'].std()
     )
print("Mean estimation in the original data",
      iris['sepal length (cm)'].mean(),
      "\nSD estimation in the original data",
      iris['sepal length (cm)'].std()
     )

Mean estimation in MCAR data with pairwise deletion 5.884057971014491 
SD estimation in MCAR data with pairwise deletion 0.8735699555148263
Mean estimation in MAR data with pairwise deletion 5.878095238095238 
SD estimation in MAR data with pairwise deletion 0.8160624212999122
Mean estimation in the original data 5.843333333333335 
SD estimation in the original data 0.8280661279778629


# Single Imputation

we can impute (or in plain English -- "guess") the missing value with various imputation methods.

# Simple imputation

https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

For MCAR data, simple imputation produces unbiased estimation of marginal means, but underestimates variances, and overestimate correlations.

In [10]:
from sklearn.impute import SimpleImputer
iris_MAR_mean = iris_MAR.copy()

In [11]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(iris_MAR)
print(imp.transform(iris_MAR))

[[5.1        3.5        1.4        0.2       ]
 [4.9        3.         1.4        0.2       ]
 [4.7        3.2        1.3        0.2       ]
 [4.6        3.1        1.5        0.2       ]
 [5.         3.6        1.4        0.2       ]
 [5.87809524 3.9        1.7        0.4       ]
 [5.87809524 3.4        1.4        0.3       ]
 [5.         3.4        1.5        0.2       ]
 [4.4        2.9        1.4        0.2       ]
 [4.9        3.1        1.5        0.1       ]
 [5.4        3.7        1.5        0.2       ]
 [4.8        3.4        1.6        0.2       ]
 [5.87809524 3.         1.4        0.1       ]
 [5.87809524 3.         1.1        0.1       ]
 [5.8        4.         1.2        0.2       ]
 [5.7        4.4        1.5        0.4       ]
 [5.4        3.9        1.3        0.4       ]
 [5.1        3.5        1.4        0.3       ]
 [5.7        3.8        1.7        0.3       ]
 [5.87809524 3.8        1.5        0.3       ]
 [5.4        3.4        1.7        0.2       ]
 [5.1        

# Regression Imputation/Conditional Mean Imputation# Regression Imputation/Conditional Mean Imputation

You can basically view the missing values as  
Y
t
e
s
t
  and the observed values as  
Y
t
r
a
i
n
 . And use the observed completed cases to train a prediction model and then predict the missing values with the incompelte cases.

In [12]:
from sklearn import linear_model
iris_MAR_regression = iris_MAR.copy()

In [13]:
iris_MAR_regression_model = iris_MAR_regression.copy().dropna()

model = linear_model.LinearRegression()
Xs = iris_MAR_regression_model['sepal width (cm)'].values.reshape(-1, 1)
ys = iris_MAR_regression_model['sepal length (cm)'].values.reshape(-1, 1)
model.fit(X = Xs, y = ys)

null_index = iris_MAR_regression['sepal length (cm)'].isnull()

na_result = model.predict(iris_MAR_regression[null_index]['sepal width (cm)'].values.reshape(-1, 1))


iris_MAR_regression.loc[iris_MAR_regression['sepal length (cm)'].isnull(), 'sepal length (cm)'] = na_result.reshape(len(na_result),)

iris_MAR_regression

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


# Nearest neighbours or hot-deck imputation

In [14]:
from sklearn.impute import KNNImputer
iris_MAR_knn = iris_MAR.copy()

In [15]:
knn_imputer = KNNImputer(n_neighbors=2, weights="uniform")

knn_imputer.fit_transform(iris_MAR_knn)

array([[5.1 , 3.5 , 1.4 , 0.2 ],
       [4.9 , 3.  , 1.4 , 0.2 ],
       [4.7 , 3.2 , 1.3 , 0.2 ],
       [4.6 , 3.1 , 1.5 , 0.2 ],
       [5.  , 3.6 , 1.4 , 0.2 ],
       [5.4 , 3.9 , 1.7 , 0.4 ],
       [5.15, 3.4 , 1.4 , 0.3 ],
       [5.  , 3.4 , 1.5 , 0.2 ],
       [4.4 , 2.9 , 1.4 , 0.2 ],
       [4.9 , 3.1 , 1.5 , 0.1 ],
       [5.4 , 3.7 , 1.5 , 0.2 ],
       [4.8 , 3.4 , 1.6 , 0.2 ],
       [4.9 , 3.  , 1.4 , 0.1 ],
       [4.55, 3.  , 1.1 , 0.1 ],
       [5.8 , 4.  , 1.2 , 0.2 ],
       [5.7 , 4.4 , 1.5 , 0.4 ],
       [5.4 , 3.9 , 1.3 , 0.4 ],
       [5.1 , 3.5 , 1.4 , 0.3 ],
       [5.7 , 3.8 , 1.7 , 0.3 ],
       [5.25, 3.8 , 1.5 , 0.3 ],
       [5.4 , 3.4 , 1.7 , 0.2 ],
       [5.1 , 3.7 , 1.5 , 0.4 ],
       [4.6 , 3.6 , 1.  , 0.2 ],
       [5.1 , 3.3 , 1.7 , 0.5 ],
       [5.1 , 3.4 , 1.9 , 0.2 ],
       [4.75, 3.  , 1.6 , 0.2 ],
       [5.  , 3.4 , 1.6 , 0.4 ],
       [5.2 , 3.5 , 1.5 , 0.2 ],
       [5.2 , 3.4 , 1.4 , 0.2 ],
       [4.7 , 3.2 , 1.6 , 0.2 ],
       [4.